### Organize the 1000+ records imported with third party resource


In [1]:
!which python

/Users/yevgeniykim/code/SchemesSG_v3/dataset_worfklow/venv/bin/python


In [2]:
# Cell 2: Load Environment Variables and Configure OpenAI Client (v1.x)
import os
import openai
from dotenv import load_dotenv
from openai import AzureOpenAI # Import the specific client

# Load environment variables from .env file
# Make sure your .env file is in the dataset_worfklow directory and contains:
# AZURE_OPENAI_ENDPOINT=your_endpoint
# AZURE_OPENAI_API_KEY=your_api_key
# OPENAI_API_VERSION=your_api_version
# AZURE_OPENAI_CHAT_DEPLOYMENT=your_chat_deployment_name

# Construct the path to the .env file relative to the notebook location
dotenv_path = os.path.join(os.path.dirname(__file__), '.env') if '__file__' in locals() else '.env'
loaded = load_dotenv(dotenv_path=dotenv_path)
print(f".env file found and loaded: {loaded}")

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")

# Check if environment variables are loaded
if not all([azure_endpoint, api_key, api_version, chat_deployment]):
    print(f"Attempted to load .env from: {os.path.abspath(dotenv_path)}")
    print(f"Loaded variables: endpoint={azure_endpoint is not None}, key={api_key is not None}, version={api_version is not None}, deployment={chat_deployment is not None}")
    raise ValueError("Azure OpenAI environment variables not set properly. Please check your .env file in the 'dataset_worfklow' directory.")

# Configure the AzureOpenAI client (v1.x syntax)
client = AzureOpenAI(
    azure_endpoint = azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

print("AzureOpenAI client configured.")
# Note: client.base_url includes the full path, potentially including deployment which isn't always desired for display
print(f"Using Endpoint: {azure_endpoint}")
print(f"Using API Version: {api_version}")
print(f"Using Chat Deployment: {chat_deployment}")


.env file found and loaded: True
AzureOpenAI client configured.
Using Endpoint: https://bettersg-openai-norwayeast-prod.openai.azure.com/
Using API Version: 2024-08-01-preview
Using Chat Deployment: gpt-4o-mini


In [3]:
def call_azure_openai(prompt: str, deployment_name: str):
    """Calls the Azure OpenAI chat completion API using openai v1.x.

    Args:
        prompt: The user prompt to send to the model.
        deployment_name: The name of the Azure OpenAI deployment (model).

    Returns:
        The response object from the OpenAI API or None if an error occurs.
    """
    try:
        # Use the client object and the new method signature
        response = client.chat.completions.create(
            model=deployment_name, # Use 'model' instead of 'engine'
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150
        )
        return response # The response object structure is also different in v1.x
    # Updated error handling for v1.x
    except openai.AuthenticationError as e:
        print(f"Authentication Error: {e.body['message']}") # Access error details differently
        print("Please check your AZURE_OPENAI_API_KEY and endpoint.")
    except openai.RateLimitError as e:
        print(f"Rate Limit Error: {e.body['message']}")
    except openai.BadRequestError as e: # Covers InvalidRequestError and others
         print(f"Bad Request Error: {e.body['message']}")
         print(f"Please check your deployment name ('{deployment_name}'), API version ('{api_version}'), and prompt content.")
    except openai.APIConnectionError as e:
        print(f"API Connection Error: {e}")
        print(f"Could not connect to the Azure endpoint: {client.base_url}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        # Optionally print the full exception for debugging
        # import traceback
        # traceback.print_exc()
    return None

In [4]:
# Cell 4: Test the Function (v1.x)
# Simple test
test_prompt = "Explain the difference between Azure OpenAI and OpenAI API in simple terms."
print(f"--- Sending prompt to deployment '{chat_deployment}' ---")
print(f"Prompt: {test_prompt}")

# Ensure the client is available (defined in Cell 2)
if 'client' not in locals():
     print("Error: AzureOpenAI client not configured. Please run Cell 2 first.")
else:
    response = call_azure_openai(test_prompt, chat_deployment)

    print("\n--- Response ---")
    if response:
        # Access the response content using the v1.x structure
        try:
            # The response object is now a pydantic model
            print(response.choices[0].message.content)
        except (AttributeError, IndexError, TypeError) as e:
            print(f"Could not extract content from response: {e}")
            print("Full Response Object:")
            print(response) # Print the pydantic model object
    else:
        print("Failed to get a response.")


--- Sending prompt to deployment 'gpt-4o-mini' ---
Prompt: Explain the difference between Azure OpenAI and OpenAI API in simple terms.

--- Response ---
Sure! Here’s a simple explanation of the difference between Azure OpenAI and the OpenAI API:

### OpenAI API:
- **What it is:** This is a direct service provided by OpenAI that allows developers to access its AI models, like ChatGPT, DALL-E, and others.
- **Access:** You can use it by signing up on the OpenAI website and getting an API key.
- **Focus:** It’s focused on providing a straightforward way to integrate AI capabilities into applications, with various models and tools available directly from OpenAI.

### Azure OpenAI:
- **What it is:** This is a service offered by Microsoft through its Azure cloud platform that provides access to OpenAI’s models.
- **Integration:** It allows


In [5]:
## Reference data
who_is_it_for = [
  "Children",
  "Youth",
  "Youth-at-risk",
  "Teenagers facing pregnancy",
  "Young adults",
  "Students",
  "Families",
  "Single parents",
  "Women",
  "Pregnant individuals in distress",
  "Elderly",
  "Elderly with dementia",
  "Persons with disabilities (PWDs)",
  "Persons with special needs",
  "Persons on autism spectrum",
  "Persons with chronic or terminal illnesses",
  "Persons with mental health issues",
  "Caregivers",
  "Low income",
  "Low income families",
  "Low income elderly",
  "Unemployed",
  "Retrenched",
  "Homeless",
  "Need shelter",
  "Need food support",
  "Foreign domestic workers/maids",
  "Migrant workers/Foreign workers",
  "Ex-offenders",
  "Inmates",
  "Families of inmates or ex-offenders",
  "Victims of abuse or harassment",
  "Facing end of life",
  "Facing financial hardship",
  "Need mortgage support",
  "Individuals needing legal aid",
  "Individuals struggling with loss",
  "Individuals with gambling addiction",
  "Transnational families/Foreign spouses",
  "Malay/Muslim community",
  "Indian community",
  "Chinese community",
  "General public"
]

what_it_gives = [
  "Counselling",
  "Casework",
  "Emotional care",
  "Mental health assessment and treatment",
  "Psychological support/Psychotherapy",
  "Befriending services",
  "Helpline services",
  "Referral services",
  "Educational programmes",
  "Vocational training",
  "Employment assistance",
  "Skills training and job matching",
  "Financial assistance (general)",
  "Financial assistance for daily living expenses",
  "Financial assistance for healthcare",
  "Financial assistance for chronic or terminal illnesses",
  "Financial assistance for education",
  "Financial assistance for kindergarten/student care",
  "Financial assistance for assistive technology and medical equipment",
  "Financial assistance for housing",
  "Mortgage assistance",
  "Debt assistance",
  "Burial and emergency assistance",
  "Food support",
  "Housing/Shelter",
  "Respite care/Caregiver support",
  "Elder sitting and caregiving services",
  "Child protection services",
  "Babysitting/Childcare services",
  "Student care",
  "Tuition/Enrichment programmes",
  "Transport subsidies",
  "Medical transport assistance",
  "Healthcare (general/basic services)",
  "Dental services",
  "Traditional Chinese Medicine (TCM)",
  "Rehabilitation services (Physiotherapy/Occupational therapy)",
  "Legal aid and services",
  "Protection against violence",
  "Residential care/programmes",
  "Addictions treatment and rehabilitation",
  "Support groups",
  "Bereavement support",
  "End-of-life care",
  "Identification and safety tagging",
  "Subsidies for Foreign Domestic Workers (FDWs)",
  "Retirement and financial planning assistance",
  "Benefits and perks for PWDs (transport, discounts, facilities)",
  "Technology assistance (internet/computers)",
  "Home retrofitting and assistive technology",
  "Funding for community projects",
  "COVID-19 support",
  "Information services"
]

scheme_type = [
  "Low Income",
  "Family",
  "Children",
  "Youth",
  "Youth-at-Risk",
  "Women",
  "Single Parents",
  "Elderly",
  "Caregiver Support",
  "Persons with Disabilities (PWD)",
  "Special Needs",
  "Ex-offender Support",
  "Incarcerated/Inmate Family Support",
  "Education Support",
  "Student Care Support",
  "Healthcare",
  "Mental Health",
  "End-of-Life/Palliative Care",
  "Chronic or Terminal Illness Support",
  "Food Support",
  "Housing/Shelter",
  "Homelessness Support",
  "Employment Support",
  "Vocational Training",
  "Debt Assistance",
  "Financial Assistance",
  "Transport Support",
  "Technology Support",
  "Legal Aid",
  "Abuse/Family Violence",
  "Protection from Violence",
  "Loss of Breadwinner Support",
  "COVID-19 Support",
  "Foreign Domestic Workers (FDWs)",
  "Migrant Workers/Foreign Workers",
  "Foreign Spouse/Transnational Family Support",
  "Community Funding",
  "Residential Care",
  "Addictions Rehabilitation",
  "Counselling and Emotional Support",
  "Mental Health Rehabilitation",
  "Dental Healthcare",
  "Traditional Chinese Medicine (TCM)",
  "Elderly Housing and Home Improvement",
  "General Healthcare Subsidies",
  "Financial Planning and Retirement Support",
  "Referral and Information Services",
  "General Public Support"
]

scheme_type_two_level = {
  "Family and Community Support": [
    "Family",
    "Children",
    "Youth",
    "Youth-at-Risk",
    "Women",
    "Single Parents",
    "Loss of Breadwinner Support",
    "Foreign Spouse/Transnational Family Support",
    "Incarcerated/Inmate Family Support",
    "General Public Support"
  ],
  "Financial and Employment Assistance": [
    "Low Income",
    "Financial Assistance",
    "Debt Assistance",
    "Employment Support",
    "Vocational Training",
    "Financial Planning and Retirement Support",
    "COVID-19 Support"
  ],
  "Healthcare and Medical Support": [
    "Healthcare",
    "Chronic or Terminal Illness Support",
    "Dental Healthcare",
    "Traditional Chinese Medicine (TCM)",
    "General Healthcare Subsidies"
  ],
  "Mental Health and Emotional Care": [
    "Mental Health",
    "Counselling and Emotional Support",
    "Mental Health Rehabilitation",
    "Addictions Rehabilitation"
  ],
  "Elderly and Caregiving Support": [
    "Elderly",
    "Caregiver Support",
    "Elderly Housing and Home Improvement",
    "End-of-Life/Palliative Care"
  ],
  "Special Needs and Disability Support": [
    "Persons with Disabilities (PWD)",
    "Special Needs",
    "Technology Support",
    "Transport Support"
  ],
  "Housing and Shelter": [
    "Housing/Shelter",
    "Homelessness Support"
  ],
  "Protection and Legal Support": [
    "Legal Aid",
    "Abuse/Family Violence",
    "Protection from Violence"
  ],
  "Migrant and Foreign Workers Support": [
    "Foreign Domestic Workers (FDWs)",
    "Migrant Workers/Foreign Workers"
  ],
  "Community Development and Funding": [
    "Community Funding",
    "Referral and Information Services"
  ]
}

search_booster = [
  "social service",
  "individuals",
  "families",
  "casework",
  "counselling",
  "see doctor",
  "cancer",
  "oncology",
  "hungry",
  "have not eaten",
  "meal",
  "need food",
  "child needs food",
  "rehabilitation",
  "occupational therapy",
  "physiotherapy",
  "respite services",
  "wheelchair accessible vans",
  "physical disability",
  "caregiver burden",
  "caregiver overloaded",
  "transport for disabled",
  "ex-convict need job",
  "skills training",
  "learning programme",
  "AIDS",
  "HIV positive",
  "medicine",
  "hospitalisation fees",
  "healthcare subsidy",
  "no money to see doctor",
  "caregiver burnout",
  "tired caregiver",
  "lonely",
  "sole caregiver",
  "dementia",
  "elderly depression",
  "special needs caregiving",
  "schizophrenia",
  "mentally ill",
  "alzheimer",
  "self harm",
  "mental health",
  "medical bills",
  "elderly disabled",
  "financial assistance elderly",
  "medical insurance elderly",
  "caregiver training",
  "hire helper for elderly",
  "domestic worker",
  "maid",
  "subsidized home care",
  "elderly ADL",
  "poor elderly healthcare",
  "assistive device",
  "wheelchair subsidy",
  "learn english foreign worker",
  "financial planning foreign worker",
  "daily needs financial assistance",
  "mental illness caregiver",
  "identify dementia elderly",
  "family member in jail",
  "family counselling inmate",
  "art therapy",
  "free art class",
  "migrant worker support",
  "hospice care",
  "palliative care",
  "dying elderly",
  "intellectually disabled",
  "down syndrome",
  "sexual abuse",
  "sexual assault",
  "domestic violence",
  "marital abuse",
  "divorce support",
  "teenage pregnancy",
  "underaged pregnancy",
  "STI counselling",
  "abortion counselling",
  "autism spectrum",
  "learning disability",
  "homeless",
  "no accommodation",
  "temporary housing",
  "stress",
  "anxiety",
  "interpersonal relationships",
  "grief support",
  "bereavement",
  "anger management",
  "free TCM",
  "child protection",
  "family violence",
  "gambling addiction",
  "debt problem",
  "loan shark",
  "integration foreigner",
  "new to singapore",
  "meet locals",
  "drug addiction",
  "rehab counselling",
  "leukaemia financial assistance",
  "bone marrow transplant",
  "youth gangs",
  "residential home youth",
  "halfway house ex-convict",
  "dialysis subsidy",
  "education low income",
  "student financial aid",
  "tuition subsidy",
  "student bursary",
  "kindergarten subsidy",
  "employment assistance",
  "job retraining",
  "childcare subsidy",
  "food ration",
  "COVID-19 financial assistance",
  "job lost retrenched",
  "retirement financial planning",
  "bullying",
  "cyberbullying",
  "inmate family bonding",
  "chronic illness children",
  "childhood cancer",
  "primary school registration",
  "enrichment classes",
  "mental health assessment",
  "psychiatric help",
  "anonymous counselling",
  "schizophrenia counselling",
  "bipolar disorder",
  "loss of breadwinner",
  "transport allowance ex-offender",
  "women empowerment",
  "babysitting services",
  "special needs education",
  "dyslexia help",
  "preschool financial help",
  "sandwich generation caregiver",
  "domestic abuse teen shelter",
  "free legal aid",
  "employment mediation",
  "motor accident legal help",
  "workplace injury migrant worker",
  "salary dispute foreign worker",
  "housing elderly retrofitting",
  "mortgage assistance",
  "reverse mortgage",
  "hdb grants",
  "condo mortgage",
  "buy hdb foreign spouse",
  "internet subsidy",
  "affordable computers",
  "digital inclusion",
  "malay muslim counselling",
  "elderly caregiver support",
  "senior day care",
  "marriage counselling",
  "stepfamily support",
  "foster child support",
  "adoption counselling",
  "early intervention children",
  "hoarding counselling",
  "single mother support",
  "milk diapers subsidy",
  "suicidal thoughts",
  "disability discounts",
  "car park label PWD",
  "transport subsidy disabled",
  "tattoo removal ex-offender",
  "rare disease children",
  "medical equipment subsidy",
  "stroke heart disease",
  "community project funding",
  "medical transport aid",
  "indian community support",
  "indian elderly day care",
  "group therapy",
  "media addiction",
  "cyber wellness",
  "intellectual disability adults",
  "mental health residential care",
  "step down psychiatric care",
  "radiology subsidy",
  "caregiver respite",
  "health screening subsidy",
  "preschool bursary malay muslim",
  "low wage worker training",
  "jobless retrenched",
  "urgent financial help",
  "comcare",
  "family conflict mediation",
  "end of life care",
  "will writing legacy planning",
  "bereavement sudden death",
  "dialysis treatment",
  "family counselling indian community",
  "migrant worker covid mental health",
  "elder abuse",
  "spousal violence",
  "public protection order PPO",
  "child abuse hotline",
  "special needs trust fund",
  "special needs caregiver legacy planning",
  "migrant worker human trafficking",
  "depression support",
  "scholarship madrasah",
  "credit card debt",
  "bankruptcy help",
  "preschool subsidy",
  "special needs transport",
  "PWD driving subsidy",
  "food vouchers",
  "financial hardship",
  "hoarding elderly",
  "family bereavement muslim",
  "parental control issues",
  "teenage angst counselling",
  "mentoring youth at risk",
  "recalcitrant youth tuition",
  "caregiver stress hotline",
  "adult disability hostel",
  "elderly isolation befriending",
  "gambling debt",
  "elderly chronic illness financial help"
]


In [6]:
# Cell 5: Load and process scheme data
import pandas as pd
import numpy as np # Often useful with pandas

# Define column names
column_names = [
    'Scheme', 'Agency', 'Description', 'Image', 'Link', "Who's it for",
    'What it gives', 'Scheme Type', 'search_booster', 'phone', 'email',
    'address', 'How to apply'
]

# Define the path to the CSV file
csv_file_path = 'schemesv3_2025_expanded.csv'

# Load the entire CSV
try:
    # --- MODIFICATION HERE ---
    # Add on_bad_lines='warn' to report and skip problematic lines
    full_df = pd.read_csv(
        csv_file_path,
        names=column_names,
        header=0,              # Still assuming row 0 is the header
        on_bad_lines='warn'    # Skip lines with too many fields, print a warning
    )
    # --- END MODIFICATION ---

    print(f"Successfully loaded {len(full_df)} rows from {csv_file_path} (after handling potential bad lines).")

except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
    full_df = pd.DataFrame(columns=column_names)
except Exception as e:
    print(f"An error occurred during CSV loading: {e}")
    full_df = pd.DataFrame(columns=column_names) # Ensure full_df exists even on other errors


# Check if loading was successful before proceeding
if not full_df.empty:
    # Define the split point
    split_index = 302

    # Split the DataFrame
    # Ensure split_index is within bounds
    if split_index > len(full_df):
        print(f"Warning: split_index ({split_index}) is larger than the number of loaded rows ({len(full_df)}). Adjusting split.")
        existing_df = full_df.copy()
        incoming_df = pd.DataFrame(columns=column_names) # No incoming rows if split is beyond end
    else:
        existing_df = full_df.iloc[:split_index].copy()
        incoming_df = full_df.iloc[split_index:].copy()

    print(f"Split data: {len(existing_df)} existing rows, {len(incoming_df)} incoming rows.")

    # Create an empty DataFrame for dropped rows
    dropped_df = pd.DataFrame(columns=column_names)

    # --- Filter incoming_df based on links in existing_df ---

    # Get the set of unique, non-null links from existing_df for efficient checking
    existing_links = set(existing_df['Link'].dropna().unique())
    print(f"Found {len(existing_links)} unique links in existing_df.")

    # Identify rows in incoming_df whose 'Link' is already in existing_links
    # Handle potential NaN values in 'Link' column - they should not be considered duplicates
    is_duplicate_link = incoming_df['Link'].isin(existing_links) & incoming_df['Link'].notna()

    # Select the duplicate rows for dropped_df
    dropped_rows = incoming_df[is_duplicate_link]
    dropped_df = pd.concat([dropped_df, dropped_rows], ignore_index=True)

    # Select the non-duplicate rows to keep in incoming_df
    incoming_df = incoming_df[~is_duplicate_link].reset_index(drop=True)

    print(f"Processing complete:")
    print(f"- {len(incoming_df)} rows remaining in incoming_df.")
    print(f"- {len(dropped_df)} duplicate rows moved to dropped_df.")

    # Display the first few rows of each resulting DataFrame (optional)
    print("\n--- Existing DF Head ---")
    print(existing_df.head())
    print("\n--- Incoming DF Head (After Filtering) ---")
    print(incoming_df.head())
    print("\n--- Dropped DF Head ---")
    print(dropped_df.head())
else:
    print("CSV loading failed or resulted in an empty DataFrame. Cannot proceed with splitting and filtering.")


Successfully loaded 1474 rows from schemesv3_2025_expanded.csv (after handling potential bad lines).
Split data: 302 existing rows, 1172 incoming rows.
Found 274 unique links in existing_df.
Processing complete:
- 1103 rows remaining in incoming_df.
- 69 duplicate rows moved to dropped_df.

--- Existing DF Head ---
                                     Scheme  \
0                          Various services   
1                      Financial Assistance   
2                           Food Assistance   
3  Rehabilitation Centre and Respite Centre   
4                          Various Services   

                                              Agency  \
0                       Montfort Care Family Service   
1             365 Cancer Prevention Society (365CPS)   
2                                   A Packet of Rice   
3  Abilities Beyond Limitations and Expectations ...   
4                   Acronis Cyber Foundation Program   

                                         Description  \
0  Prov

/var/folders/xv/8hpschc94mq0cfdl4h76xmtc0000gn/T/ipykernel_99710/2082690705.py:19: ParserWarning: Skipping line 930: expected 13 fields, saw 24

  full_df = pd.read_csv(


In [7]:
import json
import pandas as pd

# --- Ensure Reference Data is Accessible ---
# These lists should have been defined in a previous cell.
# If not, uncomment and paste them here or ensure the previous cell is run.
# who_is_it_for = [...]
# what_it_gives = [...]
# scheme_type = [...]
# search_booster = [...]

# Create a dictionary to hold the reference data for easier passing
reference_data = {
    "WHO_IS_IT_FOR_OPTIONS": who_is_it_for,
    "WHAT_IT_GIVES_OPTIONS": what_it_gives,
    "SCHEME_TYPE_OPTIONS": scheme_type,
    "SEARCH_BOOSTER_EXAMPLES": search_booster # Use as examples/inspiration
}

# --- Helper function to format lists for the prompt ---
def format_list_for_prompt(list_name: str, items: list) -> str:
    """Formats a list into a string suitable for the LLM prompt."""
    items_str = "\n - ".join(items)
    return f"--- {list_name} ---\n - {items_str}\n"

# --- Function to process one row ---
def populate_row_with_llm(row: pd.Series, ref_data: dict) -> dict:
    """
    Uses Azure OpenAI to populate 'Who's it for', 'What it gives',
    'Scheme Type', and 'search_booster' for a given row.

    Args:
        row: A pandas Series representing a row from the DataFrame.
        ref_data: A dictionary containing the reference lists.

    Returns:
        A dictionary containing the generated values for the four columns,
        or None if an error occurred.
    """
    # Prepare the reference data strings for the prompt
    who_options_str = format_list_for_prompt("WHO_IS_IT_FOR_OPTIONS", ref_data["WHO_IS_IT_FOR_OPTIONS"])
    what_options_str = format_list_for_prompt("WHAT_IT_GIVES_OPTIONS", ref_data["WHAT_IT_GIVES_OPTIONS"])
    type_options_str = format_list_for_prompt("SCHEME_TYPE_OPTIONS", ref_data["SCHEME_TYPE_OPTIONS"])
    booster_examples_str = format_list_for_prompt("SEARCH_BOOSTER_EXAMPLES (Keywords for search)", ref_data["SEARCH_BOOSTER_EXAMPLES"])

    # Extract relevant fields from the row
    scheme_name = row.get('Scheme', 'N/A')
    agency = row.get('Agency', 'N/A')
    description = row.get('Description', '') # Description might be long, ensure it's included
    link = row.get('Link', 'N/A')

    # Construct the prompt
    # Place reference data first for potential caching benefits
    prompt = f"""\
{who_options_str}
{what_options_str}
{type_options_str}
{booster_examples_str}
--- TASK ---
Analyze the following scheme details:
Scheme: {scheme_name}
Agency: {agency}
Description: {description}
Link: {link}

Based ONLY on the scheme details provided and referring to the OPTIONS/EXAMPLES lists above, determine the most relevant values for the following categories.

Format your response STRICTLY as a JSON object containing EXACTLY these four keys: "Who's it for", "What it gives", "Scheme Type", "search_booster".

- For "Who's it for", "What it gives", and "Scheme Type", select the most relevant items DIRECTLY from the corresponding OPTIONS lists. Return them as a single comma-separated string for each key (e.g., "Low income, Elderly"). If none seem relevant, return an empty string "" for that key. Limit selections to a maximum of 5 per category if possible.
- For "search_booster", generate a comma-separated string of relevant search keywords inspired by the scheme details and the SEARCH_BOOSTER_EXAMPLES. Aim for 5-10 keywords.

Example JSON Output:
{{
  "Who's it for": "Elderly, Low income families",
  "What it gives": "Financial assistance for healthcare, Befriending services",
  "Scheme Type": "Elderly, Healthcare, Low Income",
  "search_booster": "senior care, medical subsidy, poor elderly help, hospital bill assistance, befriender volunteer"
}}

Return ONLY the JSON object.

JSON Response:
"""

    print(f"--- Sending Prompt for Scheme: {scheme_name} ---")
    # print(prompt) # Uncomment to debug the full prompt

    # Call the LLM
    # Make sure 'client' and 'chat_deployment' are defined from previous cells
    if 'client' not in globals() or 'chat_deployment' not in globals():
         print("Error: AzureOpenAI client or deployment not configured.")
         return None

    response = call_azure_openai(prompt, chat_deployment)

    if response:
        try:
            content = response.choices[0].message.content
            # Clean the response: Sometimes LLMs add markdown backticks or "```json"
            if content.startswith("```json"):
                content = content[7:]
            if content.endswith("```"):
                content = content[:-3]
            content = content.strip()

            print(f"--- Raw LLM Response for {scheme_name} ---")
            print(content)

            # Parse the JSON response
            parsed_data = json.loads(content)

            # Validate the expected keys are present
            expected_keys = ["Who's it for", "What it gives", "Scheme Type", "search_booster"]
            if all(key in parsed_data for key in expected_keys):
                print(f"--- Successfully Parsed Response for {scheme_name} ---")
                return parsed_data
            else:
                print(f"Error: LLM response for {scheme_name} missing expected JSON keys.")
                print(f"Expected: {expected_keys}, Got: {list(parsed_data.keys())}")
                return None

        except json.JSONDecodeError as e:
            print(f"Error: Failed to parse JSON response for scheme '{scheme_name}': {e}")
            print(f"Problematic Content: {content}")
            return None
        except (AttributeError, IndexError, TypeError) as e:
            print(f"Error: Could not extract content from LLM response object for scheme '{scheme_name}': {e}")
            print(f"Full Response Object: {response}")
            return None
    else:
        print(f"Error: Failed to get LLM response for scheme '{scheme_name}'.")
        return None


In [8]:

# --- Test with the first row of incoming_df ---
if not incoming_df.empty:
    print("\n--- Testing with the first row of incoming_df ---")
    first_row = incoming_df.iloc[0]
    generated_values = populate_row_with_llm(first_row, reference_data)

    if generated_values:
        print("\n--- Generated Values for First Row ---")
        print(json.dumps(generated_values, indent=2))
    else:
        print("\n--- Failed to generate values for the first row ---")
else:
    print("\n--- Skipping test: incoming_df is empty. ---")


--- Testing with the first row of incoming_df ---
--- Sending Prompt for Scheme: Blossom Kaki (Senior Volunteerism) Programme ---
--- Raw LLM Response for Blossom Kaki (Senior Volunteerism) Programme ---
{
  "Who's it for": "Elderly, Caregivers, General public",
  "What it gives": "Befriending services, Volunteer opportunities, Emotional care, Community participation",
  "Scheme Type": "Elderly, Caregiver Support, Community Funding",
  "search_booster": "senior volunteerism, elderly support, community care for seniors, active aging, social harmony"
}
--- Successfully Parsed Response for Blossom Kaki (Senior Volunteerism) Programme ---

--- Generated Values for First Row ---
{
  "Who's it for": "Elderly, Caregivers, General public",
  "What it gives": "Befriending services, Volunteer opportunities, Emotional care, Community participation",
  "Scheme Type": "Elderly, Caregiver Support, Community Funding",
  "search_booster": "senior volunteerism, elderly support, community care for seni

In [10]:
# Cell: Save existing_df to CSV

output_csv_path = 'schemesv3_2025_processed.csv'
print(f"\n--- Attempting to save existing_df to {output_csv_path} ---")

try:
    # Check if existing_df exists and is not empty
    if 'existing_df' in locals() and not existing_df.empty:
        existing_df.to_csv(output_csv_path, index=False, encoding='utf-8')
        print(f"Successfully saved {len(existing_df)} rows to {output_csv_path}")
    elif 'existing_df' not in locals():
         print("Error: existing_df does not exist. Please ensure previous cells have run successfully.")
    else: # existing_df exists but is empty
         print("Warning: existing_df is empty. Saving an empty file.")
         existing_df.to_csv(output_csv_path, index=False, encoding='utf-8')

except Exception as e:
    print(f"Error saving DataFrame to CSV: {e}")



--- Attempting to save existing_df to schemesv3_2025_processed.csv ---
Successfully saved 302 rows to schemesv3_2025_processed.csv


In [12]:
# Cell: Iterate, process, and append incoming_df rows to CSV

import time # To add delays if needed
import pandas as pd # Ensure pandas is imported

# --- Iterate and Append incoming_df ---

# Make sure the output path is defined (should be from the cell above)
if 'output_csv_path' not in locals():
    output_csv_path = 'schemesv3_2025_processed.csv'
    print(f"Warning: output_csv_path not found, defaulting to {output_csv_path}")


print(f"\n--- Starting LLM processing and appending for {len(incoming_df)} rows ---")

# Define how often to print progress updates
progress_interval = 25 # Print update every 25 rows (adjust as needed)
total_rows = len(incoming_df)
rows_appended_count = 0

# Iterate through the DataFrame indices
for i, index in enumerate(incoming_df.index): # Use enumerate to get a counter
    # Get a copy of the row to update
    row_to_process = incoming_df.loc[index].copy()

    # Print progress update periodically
    if (i + 1) % progress_interval == 0 or i == 0 or i == total_rows - 1:
         print(f"--- Processing row {i+1}/{total_rows} (Index: {index}) ---")

    # print(f"\nProcessing row index {index}: Scheme '{row_to_process.get('Scheme', 'N/A')}'") # Optional detailed print

    # Call the function to get the generated values
    generated_values = populate_row_with_llm(row_to_process, reference_data)

    if generated_values:
        # Update the row Series with the new values
        try:
            row_to_process["Who's it for"] = generated_values.get("Who's it for", '')
            row_to_process["What it gives"] = generated_values.get("What it gives", '')
            row_to_process["Scheme Type"] = generated_values.get("Scheme Type", '')
            row_to_process["search_booster"] = generated_values.get("search_booster", '')

            # --- Append the updated row to the CSV ---
            # Convert the updated Series to a DataFrame (1 row)
            row_df_to_append = pd.DataFrame([row_to_process])

            # Append using mode='a' and header=False
            row_df_to_append.to_csv(output_csv_path, mode='a', header=False, index=False, encoding='utf-8')
            rows_appended_count += 1
            # print(f"Successfully appended processed row index {index} to CSV.") # Optional confirmation

            # Optional: Also update the main incoming_df in memory if you need it later
            incoming_df.loc[index, ["Who's it for", "What it gives", "Scheme Type", "search_booster"]] = [
                 row_to_process["Who's it for"], row_to_process["What it gives"], row_to_process["Scheme Type"], row_to_process["search_booster"]
            ]

        except Exception as e:
            print(f"Error updating or appending row index {index}: {e}")
    else:
        # Handle cases where LLM call failed
        print(f"Skipping append for row index {index} due to LLM processing error.")

    # Optional: Add a small delay to avoid hitting rate limits if necessary
    # time.sleep(0.5) # Sleep for 0.5 seconds between calls

print("\n--- LLM Processing and Appending Complete ---")
print(f"--- Total rows appended to {output_csv_path}: {rows_appended_count} ---")

# Display the head of the updated DataFrame in memory (optional)
print("\n--- Updated incoming_df Head (In Memory) ---")
print(incoming_df[["Scheme", "Who's it for", "What it gives", "Scheme Type", "search_booster"]].head())



--- Starting LLM processing and appending for 1103 rows ---
--- Processing row 1/1103 (Index: 0) ---
--- Sending Prompt for Scheme: Blossom Kaki (Senior Volunteerism) Programme ---
--- Raw LLM Response for Blossom Kaki (Senior Volunteerism) Programme ---
{
  "Who's it for": "Elderly, Families",
  "What it gives": "Befriending services, Community participation, Emotional care",
  "Scheme Type": "Elderly, Caregiver Support, Community Funding",
  "search_booster": "senior volunteerism, elderly care, community participation, social harmony, active aging"
}
--- Successfully Parsed Response for Blossom Kaki (Senior Volunteerism) Programme ---
--- Sending Prompt for Scheme: Eldersitter Programme ---
--- Raw LLM Response for Eldersitter Programme ---
{
  "Who's it for": "Elderly, Caregivers",
  "What it gives": "Befriending services, Respite care/Caregiver support",
  "Scheme Type": "Elderly, Caregiver Support",
  "search_booster": "elderly companionship, befriending programme, volunteer seni

In [13]:
# You can verify the CSV file manually or load it back to check
print("\n--- Reading tail of the processed CSV ---")
try:
    processed_df_check = pd.read_csv(output_csv_path)
    print(processed_df_check.tail())
except Exception as e:
    print(f"Could not read back CSV for verification: {e}")



--- Reading tail of the processed CSV ---
                                            Scheme                     Agency  \
1400                           Community Food Pack        Food From The Heart   
1401                           Community Shop 善粮小铺        Food From The Heart   
1402                       Food Assistance Project              Charis Centre   
1403  Food Ration / Diaper / Milk Powder Programme                        SHG   
1404              Food Ration Assistance Programme  Catholic Welfare Services   

                                            Description Image  \
1400  Community Food Pack\r\nThe Community Food Pack...   NaN   
1401  The Community Shop (善粮小铺) is a giving better i...   NaN   
1402  Charis Centre seeks to reach out to less fortu...   NaN   
1403  **Every March, June, Sept and Dec**\n\nWe will...   NaN   
1404  Catholic Welfare Services (CWS) is a non-profi...   NaN   

                                                   Link  \
1400  https://www.fo